# Portfolio Project: Safety Option

**Names:** Deniz Makul, Richard Sha, Mitchel Shen

**Due:** 11/25/2022 at 11:59pm ET on Dropbox

**What to submit:** your .ipynb (python code in jupyter notebooks) file as well as a pdf with all the output from your code.  




In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from threading import Thread
import time

In [2]:
#Getting tickers from csv file
csv_tickers = pd.read_csv('Tickers_Example.csv', header = None)

In [3]:
# Returns a list of of months that have less than 20 trading days

volume_start_date = '2022-01-01'
volume_end_date = '2022-10-31'

def remove_month(ticker):
    hist = ticker.history(start = volume_start_date, end = volume_end_date)
    
    months_to_remove = []
    for i in range(1,11,1):
        if i <= 9: #appends a 0 if single digit
            i = str(0) + str(i)
        df = hist.filter(like='2022-' + str(i), axis=0)
        if len(df.index) < 20:
            months_to_remove.append('2022-' + str(i))
    return months_to_remove

In [4]:
volume_index_drop = []

def filter_stocks(ticker, i):
    stock_data = yf.Ticker(ticker)
    stock_hist = stock_data.history(start = volume_start_date, end = volume_end_date, interval = '1mo', Threads = True)
    
    if stock_hist.empty:
        volume_index_drop.append(i)
        return
    
    stock_volume = stock_hist['Volume']
    stock_volume = stock_volume.dropna()
    stock_volume = stock_volume.drop(remove_month(stock_data))
    
    mean = (stock_volume.mean())

    if mean<200000:
        volume_index_drop.append(i)         
    elif(stock_data.info['market'] != 'us_market'):
        volume_index_drop.append(i)
    
    #volume_index_drop

In [5]:
#Drops the tickers that are delisted or have a average monthly volume from 
# January 1st to October 31st of less that 200,000

# Sending in the first ticker to determine which month to remove

thread_list = []

for i in range(len(csv_tickers)):
    ticker = csv_tickers[0][i]
    filter_thread = Thread(target = filter_stocks, args = (ticker,i,))
    thread_list.append(filter_thread)
    filter_thread.start()
    
for i in thread_list:
    i.join()

print(volume_index_drop)

- CELG: No data found, symbol may be delisted
- AGN: No data found, symbol may be delisted
- RTN: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
[16, 4, 47, 40, 56, 54, 48]


In [6]:
#Drops invalid tickers
csv_tickers.drop(index = volume_index_drop, inplace = True)
csv_tickers.reset_index(drop = True, inplace = True)
csv_tickers.head()

,0
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AIG


In [7]:
close_all = pd.DataFrame()

def stock_history(ticker):
    stock_data = yf.Ticker(ticker)
    stock_hist = stock_data.history(start = close_start, end = close_end, interval = '1d')
    stock_close = stock_hist['Close']
    stock_close = stock_close.dropna()
    close_all[ticker] = stock_close

In [8]:
# Mitchel
close_start = '2022-01-01'
close_end = '2022-10-31'

thread_list = []

for i in range(0,len(csv_tickers)):
    ticker = csv_tickers[0][i]
    filter_thread = Thread(target = stock_history, args = (ticker, ))
    thread_list.append(filter_thread)
    filter_thread.start()
    
for i in thread_list:
    i.join()
close_all.head()

,BIIB,AMZN,MON,PYPL,GOOG,ABBV,ACN,SBUX,CSCO,C,...,PEP,MO,COP,JPM,CVS,LMT,TXN,LLY,NEE,UNP
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,244.139999,170.404495,9.75,194.940002,145.074493,134.029205,406.156982,116.680000,63.160000,62.606880,...,172.979996,47.970001,73.400787,160.736511,103.606232,351.936951,190.600006,270.600403,91.155136,246.562317
2022-01-04,241.729996,167.522003,9.75,191.139999,144.416504,133.771881,403.254517,114.239998,61.250000,63.093052,...,173.229996,49.029999,76.584778,166.830002,103.735535,359.514771,190.800003,265.700836,90.180534,250.801544
2022-01-05,239.270004,164.356995,9.75,187.160004,137.653503,134.474579,396.152924,110.440002,60.279999,62.358833,...,173.820007,48.639999,75.271385,163.779999,105.247452,355.691132,186.820007,259.566437,89.235771,251.557831
2022-01-06,237.300003,163.253998,9.76,192.270004,137.550995,133.841156,377.022522,111.139999,60.919998,64.402740,...,173.860001,49.209999,78.097168,165.520004,102.044571,355.552063,186.759995,258.251923,85.297577,252.254440
2022-01-07,232.600006,162.554001,9.75,187.600006,137.004501,133.494751,369.791290,107.570000,61.130001,65.265938,...,174.080002,49.770000,80.236404,167.160004,103.636070,357.677460,179.440002,258.421234,85.934059,253.538147


In [9]:
close_all_pct = close_all.pct_change() * 100
close_all_pct.head()

,BIIB,AMZN,MON,PYPL,GOOG,ABBV,ACN,SBUX,CSCO,C,...,PEP,MO,COP,JPM,CVS,LMT,TXN,LLY,NEE,UNP
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,-0.987140,-1.691559,0.000000,-1.949319,-0.453553,-0.191991,-0.714617,-2.091192,-3.024066,0.776547,...,0.144525,2.209709,4.337815,3.790981,0.124802,2.153175,0.104930,-1.810628,-1.069168,1.719333
2022-01-05,-1.017661,-1.889309,0.000000,-2.082241,-4.682983,0.525296,-1.761070,-3.326327,-1.583675,-1.163708,...,0.340594,-0.795430,-1.714953,-1.828210,1.457473,-1.063555,-2.085952,-2.308762,-1.047635,0.301548
2022-01-06,-0.823338,-0.671098,0.102566,2.730285,-0.074469,-0.471035,-4.829045,0.633826,1.061711,3.277655,...,0.023008,1.171874,3.754126,1.062404,-3.043191,-0.039098,-0.032123,-0.506427,-4.413246,0.276918
2022-01-07,-1.980614,-0.428778,-0.102461,-2.428875,-0.397302,-0.258818,-1.917984,-3.212165,0.344719,1.340311,...,0.126539,1.137983,2.739198,0.990816,1.559612,0.597774,-3.919465,0.065561,0.746190,0.508894


In [10]:
#Expected Return
expected_return = close_all_pct.mean()
expected_return.head()

BIIB    0.125011
AMZN   -0.190174
MON     0.009427
PYPL   -0.316877
GOOG   -0.166870
dtype: float64

In [11]:
expected_return_from_0 = abs(0 - expected_return)
expected_return_from_0

expected_return_sorted = sorted(expected_return_from_0.items(), key=lambda x:x[1])

In [12]:
std_lst = close_all_pct.std()
std_lst = sorted(std_lst.items(), key=lambda x:x[1])

In [13]:
correlation_pct_stocks = pd.DataFrame()
index = []

for i in range(len(close_all_pct.columns)):
    
    correlation_temp = {}
    
    for j in range(len(close_all_pct.columns)):
        correlation_df = pd.DataFrame({
            csv_tickers[0][i]: close_all_pct[csv_tickers[0][i]]
        })
        
        correlation_df[csv_tickers[0][j]] = close_all_pct[csv_tickers[0][j]]
        correlation = correlation_df.corr()[csv_tickers[0][i]][csv_tickers[0][j]]

        correlation_temp[csv_tickers[0][j]] = correlation
        
    correlation_pct_stocks[csv_tickers[0][i]] = list(correlation_temp.values())
    index.append(csv_tickers[0][i])


correlation_pct_stocks.set_index([pd.Index(index)])
correlation_pct_stocks.head()

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
0,1.000000,0.196115,0.607975,0.788189,0.564777,0.661361,0.640823,0.576638,0.566142,0.298499,...,0.208881,0.370039,0.596782,0.378568,0.506691,0.720272,0.518906,0.474990,0.536375,0.471165
1,0.196115,1.000000,0.448482,0.310854,0.300005,0.195382,0.254210,0.103088,0.287487,0.222762,...,0.168036,0.369601,0.246472,0.277107,0.259999,0.235616,0.523391,0.242589,0.201065,0.368628
2,0.607975,0.448482,1.000000,0.685228,0.530735,0.486366,0.580548,0.428488,0.563878,0.328822,...,0.117035,0.438613,0.513648,0.367974,0.422302,0.523594,0.591601,0.490681,0.468793,0.542299
3,0.788189,0.310854,0.685228,1.000000,0.600418,0.677280,0.649058,0.548413,0.589600,0.384701,...,0.203834,0.411081,0.640848,0.365729,0.547363,0.735174,0.581976,0.556123,0.574278,0.527163
4,0.564777,0.300005,0.530735,0.600418,1.000000,0.456730,0.737495,0.534551,0.757113,0.241459,...,0.395121,0.290292,0.601893,0.372322,0.340778,0.489535,0.400360,0.512507,0.509901,0.732907


In [14]:
correlation_pct_mean = pd.DataFrame()
correlation_pct_mean = correlation_pct_stocks.sum()
correlation_pct_mean = correlation_pct_mean - 1
correlation_pct_mean = correlation_pct_mean/(len(correlation_pct_stocks) - 1)

correlation_pct_mean.head()

AAPL    0.497130
ABBV    0.284643
ABT     0.467452
ACN     0.523949
AIG     0.473681
dtype: float64

In [15]:
correlation_from_0 = abs(0 - correlation_pct_mean)
correlation_from_0 = sorted(correlation_from_0.items(), key=lambda x:x[1])

In [16]:
points_system = pd.DataFrame()
expected_lst = []
std_deviation_lst = []
correlation_lst = []

for i in expected_return_sorted:
    expected_lst.append(i[0])
    
for i in std_lst:
    std_deviation_lst.append(i[0])

for i in correlation_from_0:
    correlation_lst.append(i[0])

points_system['Expected Returns'] = expected_lst
points_system['Standard Deviation'] = std_deviation_lst
points_system['Correlation'] = correlation_lst
points_system.index += 1 

points_system = points_system.set_index([points_system.index[::-1]])

points_system.head()

,Expected Returns,Standard Deviation,Correlation
55,SO,MON,MON
54,PM,BMY,LMT
53,MO,MRK,OXY
52,MON,PEP,SLB
51,T,KO,COP


In [17]:
score = pd.DataFrame()
    
score['Tickers'] = expected_lst

score['Points'] = None

for i in range(len(score)):
    score['Points'][i] = (int(points_system[points_system['Expected Returns'] == expected_lst[i]].index.values)) + (int(points_system[points_system['Standard Deviation'] == expected_lst[i]].index.values)) + (int(points_system[points_system['Correlation'] == expected_lst[i]].index.values)) 

score = score.sort_values("Points")
score = score[::-1]
score = score.reset_index(drop=True)
score.index += 1 

score.head()

,Tickers,Points
1,MON,162
2,SO,147
3,MO,144
4,PM,142
5,ABBV,134


In [18]:
number_of_stocks = 25

score_dropped = score.copy()
score_dropped.drop(labels = range(number_of_stocks + 1, len(score) + 1), inplace = True)
score_dropped.head()

,Tickers,Points
1,MON,162
2,SO,147
3,MO,144
4,PM,142
5,ABBV,134


In [19]:
min_weighting = 100/(2*number_of_stocks)
base_weighting_total = min_weighting * number_of_stocks

score_dropped['Base Weighting'] = min_weighting
weighting_sum = score_dropped['Points'].sum()

score_dropped['Points-based Weighting'] = (score_dropped['Points']/weighting_sum) * (100 - base_weighting_total)
score_dropped['Total Weighting'] = score_dropped['Base Weighting'] + score_dropped['Points-based Weighting']
score_dropped.head()

,Tickers,Points,Base Weighting,Points-based Weighting,Total Weighting
1,MON,162,2.0,2.779684,4.779684
2,SO,147,2.0,2.522306,4.522306
3,MO,144,2.0,2.47083,4.47083
4,PM,142,2.0,2.436513,4.436513
5,ABBV,134,2.0,2.299245,4.299245


In [20]:
tickers_list = score_dropped['Tickers']
Portfolio_Final = pd.DataFrame()

start_date = '2022-11-22'
end_date = '2022-11-23'

for x in tickers_list:
    stock_info = yf.Ticker(x)
    stock_hist =  stock_info.history(start = start_date, end = end_date)
    
    Portfolio_Final[x] = stock_hist['Close']
    #price_list.append(recent_history_close['2022-11-22']) 
    
Portfolio_Final = Portfolio_Final.transpose()
Portfolio_Final = Portfolio_Final.reset_index()
Portfolio_Final.index += 1

Portfolio_Final.head()

Date,index,2022-11-22 00:00:00
1,MON,10.010000
2,SO,65.949997
3,MO,45.160000
4,PM,97.160004
5,ABBV,159.789993


In [21]:
total_amount_to_spend = 500000
column_names = ['Ticker', 'Price']
Portfolio_Final.columns = column_names
#value_df = pd.DataFrame()
Portfolio_Final['Shares'] = (total_amount_to_spend*(score_dropped['Total Weighting']/100))/Portfolio_Final['Price']
Portfolio_Final['Value'] = Portfolio_Final['Shares'] * Portfolio_Final['Price']
Portfolio_Final['Weight'] = score_dropped['Total Weighting']
Portfolio_Final
#print(score_dropped['Total Weighting']/100)

,Ticker,Price,Shares,Value,Weight
1,MON,10.010000,2387.454632,23898.421414,4.779684
2,SO,65.949997,342.858705,22611.530542,4.522306
3,MO,45.160000,494.998947,22354.152368,4.47083
4,PM,97.160004,228.309655,22182.566918,4.436513
5,ABBV,159.789993,134.527981,21496.22512,4.299245
6,T,19.000000,1126.864863,21410.432395,4.282086
7,CL,76.370003,279.227955,21324.639671,4.264928
8,KO,62.349998,333.759163,20809.883322,4.161977
9,BMY,78.860001,262.795973,20724.090597,4.144818
10,PEP,184.839996,109.798352,20295.126973,4.059025


In [22]:
Stocks_Final = pd.DataFrame()

Stocks_Final['Ticker'] = Portfolio_Final['Ticker'] 
Stocks_Final['Shares'] = Portfolio_Final['Shares']

Stocks_Final

NameError: name 'Stocks_Final' is not defined